<a href="https://colab.research.google.com/github/lshaoqin/spaceship-titanic/blob/main/spacetitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

# Test running pytorch
x = torch.rand(5, 3)
print(x)


tensor([[0.5114, 0.4107, 0.0717],
        [0.3023, 0.5976, 0.9859],
        [0.3056, 0.3398, 0.9503],
        [0.3909, 0.4602, 0.4734],
        [0.3650, 0.0798, 0.6085]])


In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")

train_df.head(5)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
train_df.describe()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [6]:
train_df = train_df.drop(['PassengerId', 'Name'], axis=1)
train_df.head(5)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 755.7+ KB


In [8]:
train_df.isnull().sum().sort_values(ascending=False) # Check for missing values

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
Transported       0
dtype: int64

In [9]:
train_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = train_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)
train_df.isnull().sum().sort_values(ascending=False)

HomePlanet      201
Cabin           199
Destination     182
RoomService     181
Age             179
CryoSleep         0
VIP               0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [10]:
train_df["Transported"] = train_df["Transported"].astype(int)

In [11]:
train_df["VIP"] = train_df["VIP"].astype(int)
train_df["CryoSleep"] = train_df["CryoSleep"].astype(int)

In [12]:
train_df[["Deck", "Cabin_num", "Side"]] = train_df["Cabin"].str.split("/", expand=True)
train_df[["Deck", "Cabin_num", "Side", "RoomService", "Age"]] = train_df[["Deck", "Cabin_num", "Side", "RoomService", "Age"]].fillna(value=0)

In [13]:
try:
  train_df = train_df.drop("Cabin", axis=1)
except KeyError:
  print("Field does not exist")

In [14]:
train_df.head(5)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Cabin_num,Side
0,Europa,0,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,B,0,P
1,Earth,0,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,F,0,S
2,Europa,0,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,A,0,S
3,Europa,0,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,A,0,S
4,Earth,0,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,F,1,S


In [15]:
train_df = pd.get_dummies(train_df, columns=['HomePlanet', 'Destination', 'Deck', 'Side'])
train_df = train_df.astype(np.float16)

In [16]:
for column in train_df.columns:
  train_df[column] = (train_df[column] - train_df[column].min()) / (train_df[column].max() - train_df[column].min())

train_df.head()
train_df.isnull().sum().sort_values(ascending=False)

CryoSleep                    0
Age                          0
Side_P                       0
Side_0                       0
Deck_T                       0
Deck_G                       0
Deck_F                       0
Deck_E                       0
Deck_D                       0
Deck_C                       0
Deck_B                       0
Deck_A                       0
Deck_0                       0
Destination_TRAPPIST-1e      0
Destination_PSO J318.5-22    0
Destination_55 Cancri e      0
HomePlanet_Mars              0
HomePlanet_Europa            0
HomePlanet_Earth             0
Cabin_num                    0
Transported                  0
VRDeck                       0
Spa                          0
ShoppingMall                 0
FoodCourt                    0
RoomService                  0
VIP                          0
Side_S                       0
dtype: int64

In [17]:
def split_dataset(dataset, test_ratio=0.20):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, valid_ds_pd = split_dataset(train_df)
print("{} examples in training, {} examples in testing.".format(
    len(train_ds_pd), len(valid_ds_pd)))

7032 examples in training, 1661 examples in testing.


In [18]:
from torch.utils.data import Dataset, DataLoader
class TitanicDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx].values
        if "Transported" in self.data.columns:
          features = torch.tensor(sample[self.data.columns != "Transported"], dtype=torch.float32)
          label = torch.tensor(sample[self.data.columns == "Transported"], dtype=torch.float32)
          return features, label
        else:
          return torch.tensor(sample[self.data.columns != "Transported"], dtype=torch.float32)
train_ds = TitanicDataset(train_ds_pd)
test_ds = TitanicDataset(valid_ds_pd)

In [19]:
batch_size = 32

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [20]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(len(train_ds.data.columns) - 1, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [22]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=27, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [26]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        inputs, labels = batch
        output = model(inputs)
        loss = criterion(output, labels)  # Provide your target variable
        loss.backward()
        optimizer.step()

    # Test against test set
    model.eval()
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Test Error: 
 Accuracy: 82.1%, Avg loss: 0.399108 

Test Error: 
 Accuracy: 82.5%, Avg loss: 0.394676 

Test Error: 
 Accuracy: 82.1%, Avg loss: 0.387304 

Test Error: 
 Accuracy: 81.0%, Avg loss: 0.402390 

Test Error: 
 Accuracy: 81.8%, Avg loss: 0.398255 

Test Error: 
 Accuracy: 81.9%, Avg loss: 0.395990 

Test Error: 
 Accuracy: 81.3%, Avg loss: 0.463694 

Test Error: 
 Accuracy: 82.2%, Avg loss: 0.399775 

Test Error: 
 Accuracy: 81.8%, Avg loss: 0.464104 

Test Error: 
 Accuracy: 82.4%, Avg loss: 0.401568 



In [32]:
# Load the test dataset
test_df = pd.read_csv('test.csv')
submission_id = test_df.PassengerId

# Drop unused columns
test_df = test_df.drop(['PassengerId', 'Name'], axis=1)

# Replace NaN values with zero
test_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = test_df[['VIP', 'CryoSleep', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(value=0)

# Creating New Features - Deck, Cabin_num and Side from the column Cabin and remove Cabin
test_df[["Deck", "Cabin_num", "Side"]] = test_df["Cabin"].str.split("/", expand=True)
test_df[["Deck", "Cabin_num", "Side", "RoomService", "Age"]] = test_df[["Deck", "Cabin_num", "Side", "RoomService", "Age"]].fillna(value=0)
test_df = test_df.drop('Cabin', axis=1)

# Convert boolean to 1's and 0's
test_df['VIP'] = test_df['VIP'].astype(int)
test_df['CryoSleep'] = test_df['CryoSleep'].astype(int)

# Hot encode categorical data
test_df = pd.get_dummies(test_df, columns=['HomePlanet', 'Destination', 'Deck', 'Side'])
test_df = test_df.astype(np.float16)

# Normalise data
for column in test_df.columns:
  test_df[column] = (test_df[column] - test_df[column].min()) / (test_df[column].max() - test_df[column].min())

submission_ds = TitanicDataset(test_df)

# Create dataloader
submission_loader = DataLoader(submission_ds, batch_size=batch_size, shuffle=True)

# Make and process predictions
predictions = []
model.eval()
for idx in range(len(submission_ds)):
    features = submission_ds[idx]
    features = torch.tensor(features, dtype=torch.float32).unsqueeze(0)
    with torch.no_grad():
      output = model(features)
      predictions.append(output)

n_predictions = [(prediction > 0.5) for prediction in predictions]
output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': n_predictions})
output.head()

<ipython-input-32-2d3977d664f5>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features, dtype=torch.float32).unsqueeze(0)


,PassengerId,Transported
0,0013_01,[[tensor(False)]]
1,0018_01,[[tensor(False)]]
2,0019_01,[[tensor(True)]]
3,0021_01,[[tensor(True)]]
4,0023_01,[[tensor(True)]]


In [36]:
sample_submission_df = pd.read_csv('sample_submission.csv')
sample_submission_df['Transported'] = [bool(value) for value in n_predictions]
sample_submission_df.to_csv('submission.csv', index=False)
sample_submission_df.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
